# Importing and cleaning Junior Olympics ranking data


How has the distribution of participating teams from across the country changed since 2010? 
How have non-california teams performed since 2010?
Do the same teams dominate both girls and boys categories?

In [1]:
from tabula import read_pdf
import pandas as pd

In [2]:
import PyPDF2

In [3]:
import os

In [4]:
import re
import numpy as np

In [5]:
pd.set_option('display.max_rows', None)

## Import and clean pdf files
Import and clean using a function that works with the format of the majority of the files.

In [6]:
folder_path = "/home/jessie/Documents/Data"

files = os.listdir(folder_path)
files = [x for x in files if ('.pdf' in x ) & ~('2018' in x) & (('Boys' in x) | ('Girls' in x)) & ~('2012 Girls' in x)]

In [7]:
files

['2016 Boys.pdf',
 '2013 Girls.pdf',
 '2012 Boys.pdf',
 '2013 Boys.pdf',
 '2017 Boys.pdf',
 '2015 Girls.pdf',
 '2014 Boys.pdf',
 '2017 Girls.pdf',
 '2014 Girls.pdf',
 '2015 Boys.pdf',
 '2016 Girls.pdf']

In [8]:
def clean_df(df, filename):
    df = df.set_index('Finish')

    df = df.dropna(axis=0, how='all')

    cols = df.columns.to_series()
    cols = cols.dropna()
    filtered_cols = cols[~cols.str.contains(r'unnamed', flags=re.I)]

    df = df.dropna(axis=1, how='all')
    df.columns = filtered_cols

    #move second set of columns to top
    grps = (df.index == 'Finish').cumsum()
    dfs = []
    for grp, dfg in df.groupby(grps):
        if grp != 0:
            dfg.columns = dfg.iloc[0].values
            dfs.append(dfg)
        else: dfs.append(dfg)

    df_new = pd.concat(dfs, axis=1, sort=False)
    df_new = df_new.drop('Finish', axis=0)
    df_new = df_new.dropna(axis=0, how='all')

    cols = df_new.columns.to_series()
    cols = cols.dropna()
    filtered_cols = cols[~cols.str.contains(r'unnamed', flags=re.I)]

    df_new = df_new.loc[:, df_new.columns.notnull()]
    df_new.columns = filtered_cols

    df_new = df_new.apply(lambda col: col.str.upper())

    df_stacked = df_new.stack().reset_index()
    df_stacked = df_stacked.rename(columns = {'level_0':'Finish', 'level_1':'Category', 0:'Team'})
    df_stacked['Age'] = df_stacked['Category'].str.extract(r'([0-9]{2})')
    df_stacked['Sex'] = df_stacked['Category'].str.extract(r'\b(\w+)$')
    df_stacked = df_stacked.drop('Category', axis=1)
    df_stacked['Year'] = filename
    df_stacked['Year'] = df_stacked['Year'].str.extract('^([\w\-]+)')
    
    return df_stacked

In [9]:
data = pd.DataFrame(columns=['Finish', 'Team', 'Age', 'Sex', 'Year'])

for pdf in files:
    
    df = read_pdf(pdf, stream=True, pages='all')
    df_clean = clean_df(df,pdf)
    data = pd.concat([data, df_clean], axis=0, sort=False)

In [10]:
data['Year'].value_counts()

2015    704
2017    699
2016    642
2014    589
2013    558
2012    300
Name: Year, dtype: int64

## Import and clean remaining pdf files
A few of the files have a different format and required slightly different functions to import and clean.

In [11]:
other_files = ['2010.pdf', '2011.pdf', '2012 Girls.pdf']

In [12]:
def clean_df_other(df, filename):
    df = df.set_index('Finish')
    grps = (df.index == 'Finish').cumsum()

    dfs = []
    for grp, dfg in df.groupby(grps):
        if grp != 0:
            dfg.columns = dfg.iloc[0].values
            dfs.append(dfg)
        else: dfs.append(dfg)
        
    df_new = pd.concat(dfs, axis=1, sort=False)
    df_new = df_new.drop('Finish', axis=0)
    df_new = df_new.dropna(axis=0, how='all')

    cols = df_new.columns.to_series()
    cols = cols.dropna()
    filtered_cols = cols[~cols.str.contains(r'unnamed', flags=re.I)]
    #filtered_cols = filtered_cols[~filtered_cols.str.contains(r'21', flags=re.I)]

    df_new = df_new.dropna(axis=1, how='all')
    df_new.columns = filtered_cols

    df_new = df_new.apply(lambda col: col.str.upper())
    
    df_stacked = df_new.stack().reset_index()
    df_stacked = df_stacked.rename(columns = {'level_0':'Finish', 'level_1':'Category', 0:'Team'})
    df_stacked['Age'] = df_stacked['Category'].str.extract(r'([0-9]{2})')
    df_stacked['Sex'] = df_stacked['Category'].str.extract(r'\b(\w+)$')
    df_stacked = df_stacked.drop('Category', axis=1)
    df_stacked['Year'] = filename
    df_stacked['Year'] = df_stacked['Year'].str.extract('^([\w\-]+)')
    return df_stacked

In [13]:
for pdf in other_files:
    
    df = read_pdf(pdf, stream=True, pages='all')
    df_clean = clean_df_other(df,pdf)
    data = pd.concat([data, df_clean], axis=0, sort=False)

In [14]:
data['Year'].value_counts()

2015    704
2017    699
2016    642
2014    589
2012    559
2013    558
2011    512
2010    352
Name: Year, dtype: int64

In [15]:
files_2018 = os.listdir(folder_path)
files_2018 = [x for x in files_2018 if ('2018' in x)]

In [16]:
files_2018

['2018JOresults-14UG.pdf',
 '2018JOresults-18uB.pdf',
 '2018JOresults-10UG.pdf',
 '2018JOresults-12UG.pdf',
 '2018JOresults-18UG.pdf',
 '2018JOresults-12uB.pdf',
 '2018JOresults-16UG.pdf',
 '2018JOresults-16uB.pdf',
 '2018JOresults-10UM.pdf',
 '2018JOresults-14uB.pdf']

In [17]:
df = read_pdf('2018JOresults-14uB.pdf', pages='all', lattice=True)

In [18]:
df

,14 & Under Boys,Unnamed: 1,Unnamed: 2
0,Flight,Team\rFinish,Team Name
1,NaN,1,VANGUARD A
2,2,SOCAL BLACK A,NaN
3,3,LA PREMIER,NaN
4,4,STANFORD A,NaN
5,5,SHORE AQUATICS A,NaN
6,6,NEWPORT BEACH A,NaN
7,7,LAMORINDA A,NaN
8,8,FOOTHILL A,NaN
9,9,DEL MAR A,NaN


In [19]:
def clean_df_2018(df, filename):
    df.columns = ['Finish', 'Team', 'Blank']
    df_new = df[~df['Finish'].str.contains('Flight', na=False)]
    df_new = df_new[~df_new['Finish'].str.contains('Under', na=False)]
    
    new_finish = df_new.loc[df_new['Finish'].isnull(), 'Team']
    new_team = df_new.loc[df_new['Finish'].isnull(), 'Blank']

    df_new.loc[df_new['Finish'].isnull(), 'Team'] = new_team
    df_new.loc[df_new['Finish'].isnull(), 'Finish'] = new_finish
    df_new = df_new.drop('Blank', axis=1)
    
    df_new['Category'] = filename
    df_new['Age'] = df_new['Category'].str.extract(r'-([0-9]{2})')
    df_new['Sex'] = df_new['Category'].str.extract(r'[Uu]([GB]).')
    df_new = df_new.drop('Category', axis=1)
    df_new['Year'] = filename
    df_new['Year'] = df_new['Year'].str.extract('^([\w\-]{4})')
    
    df_new.loc[df_new['Sex'] == 'G', 'Sex'] = 'Girls'
    df_new.loc[df_new['Sex'] == 'B', 'Sex'] = 'Boys'
    
    return df_new

In [20]:
clean_df_2018(df, '2018JOresults-10UG.pdf')

,Finish,Team,Age,Sex,Year
1,1,VANGUARD A,10,Girls,2018
2,2,SOCAL BLACK A,10,Girls,2018
3,3,LA PREMIER,10,Girls,2018
4,4,STANFORD A,10,Girls,2018
5,5,SHORE AQUATICS A,10,Girls,2018
6,6,NEWPORT BEACH A,10,Girls,2018
7,7,LAMORINDA A,10,Girls,2018
8,8,FOOTHILL A,10,Girls,2018
9,9,DEL MAR A,10,Girls,2018
10,10,OCWPC A,10,Girls,2018


In [21]:
for pdf in files_2018:
    
    df = read_pdf(pdf, pages='all', lattice=True)
    df_clean = clean_df_2018(df,pdf)
    data = pd.concat([data, df_clean], axis=0, sort=False)

## Import and clean excel files
The final files are excel and require a simpler function.

In [22]:
folder_path = "/home/jessie/Documents/Data"

files_2019 = os.listdir(folder_path)
files_2019 = [x for x in files_2019 if ('.xlsx' in x)]

In [23]:
def df_from_csv(path):
    df = read_csv(path, nrows=1) # read just first line for columns
    columns = df.columns.tolist() # get the columns
    cols_to_use = columns[:len(columns)-1] # drop the last one
    df = read_csv(path, usecols=cols_to_use)
    return df

In [24]:
files_2019

['NJO 2019 Session 2 Final Placing.xlsx',
 'NJO 2019 Session 1 Final Placing.xlsx']

In [25]:
def clean_excel(df, filename):
    df_new = df.drop('Flight', axis=1)
    df_new = df_new.set_index('Finish')
    df_new = df_new.apply(lambda col: col.str.upper())

    df_stacked = df_new.stack().reset_index()
    df_stacked = df_stacked.rename(columns = {'level_1':'Category', 0:'Team'})
    df_stacked['Age'] = df_stacked['Category'].str.extract(r'([0-9]{2})')
    df_stacked['Sex'] = df_stacked['Category'].str.extract(r'\b(\w+)$')
    df_stacked = df_stacked.drop('Category', axis=1)
    df_stacked['Year'] = filename
    df_stacked['Year'] = df_stacked['Year'].str.extract('([0-9]{4})')

    return df_stacked

In [26]:
for file in files_2019:
    df = pd.read_excel(file)
    df_clean = clean_excel(df, file)
    data = pd.concat([data, df_clean], axis=0, sort=False)

In [27]:
data['Year'].value_counts()

2019    875
2018    765
2015    704
2017    699
2016    642
2014    589
2012    559
2013    558
2011    512
2010    352
Name: Year, dtype: int64

In [28]:
data.head(20)

,Finish,Team,Age,Sex,Year
0,1,LAGUNA BEACH WATER POLO,10,Girls,2016
1,1,VANGUARD AQUATICS,12,Boys,2016
2,1,LOS ANGELES PREMIER,14,Boys,2016
3,1,STANFORD WATER POLO FOUNDATION,16,Boys,2016
4,1,REGENCY WATER POLO CLUB,18,Boys,2016
5,2,SANTA BARBARA PREMIER,10,Girls,2016
6,2,CC UNITED,12,Boys,2016
7,2,SOCAL WATER POLO FOUNDATION,14,Boys,2016
8,2,SLEEPY HOLLOW AQUATICS,16,Boys,2016
9,2,680 WATER POLO,18,Boys,2016


## Cleaning Squads and Team Names
Indicating A/B/C teams, removing extra characters, 

Strings to remove from Club name: 10-18, 2010-2018, ##U, ##B, ##G, ##s, B##, G##, GIRL(S), BOY(S), COED(S), JO(s), joq, (A-E), &, Under, (Tied...), -, /, \xad, ,, ., 's

to match: aquatics, water polo, club, federation, team, wp, club, polo, summer, organization, academy, swim, and, ac, inc, foundation,

replace / with space

orange but not orange county

In [46]:
data_clean = data.copy()

In [47]:
data_clean['Team'] = data_clean['Team'].str.strip(r'\\r')
data_clean['Team'] = data_clean['Team'].str.replace('\t\r', '')
data_clean['Team'] = data_clean['Team'].str.replace('\r', ' ')

In [48]:
data_clean['og_name'] = data_clean['Team']

In [49]:
data_clean['Squad'] = data_clean['Team'].str.extract(r'\b([A-F])\b')
data_clean['Squad'] = data_clean['Squad'].fillna('A')

In [50]:
data_clean['Team'] = data_clean['Team'].str.replace('ORAGNE', 'ORANGE')

In [51]:
remove_words = ['[0-9]{2}', '[UBG][0-9]{2}', '[0-9]{2}[UBGS]' '20[0-9]{2}', 'U', 'B', 'G', 'GIRL', 'BOY', 'COED',
                'GIRLS', 'BOYS', 'COEDS', 'JO', 'JOQ', 'JOS', '[A-E]', '&', 'AND', 'UNDER', 'BLACK', 'GREEN', 'BLUE', 
                'BLU', 'RED', 'WHITE', 'YELLOW', 'PINK', 'MAROON', 'GOLD', 'SILVER', 'BRONZE', 'PURPLE', 'BLK', 'GREY']

pattern = r'\b(?:{})\b'.format('|'.join(remove_words))
data_clean['Team'] = data_clean['Team'].str.replace(pattern, '')

In [52]:
data_clean['Team'] = data_clean['Team'].str.replace('\s+', ' ', regex=True)
data_clean['Team'] = data_clean['Team'].str.replace(r'\BS\b', '')
data_clean['Team'] = data_clean['Team'].str.replace(r'\(TIED.+\)', '')
data_clean['Team'] = data_clean['Team'].str.replace(r'\(TIED.+', '')
data_clean['Team'] = data_clean['Team'].str.replace(r"[\"\'.,\\\(\)&/]", '')
data_clean['Team'] = data_clean['Team'].str.replace('\t\r', '')
data_clean['Team'] = data_clean['Team'].str.replace('xad', '')
data_clean['Team'] = data_clean['Team'].str.replace('\\xad', '')
data_clean['Team'] = data_clean['Team'].str.replace('‐', '')
data_clean['Team'] = data_clean['Team'].str.replace('-', '')
data_clean['Team'] = data_clean['Team'].str.replace(r'[0-9]{2}[UBGS]', '')
data_clean['Team'] = data_clean['Team'].str.replace(r'20[0-9]{2}', '')
data_clean['Team'] = data_clean['Team'].str.replace('ORANGE COUNTY', 'ORANGECOUNTY')
data_clean['Team'] = data_clean['Team'].str.replace(r'\b(ORANGE)\b', '')
data_clean['Team'] = data_clean['Team'].str.replace('\s+', ' ', regex=True)
data_clean['Team'] = data_clean['Team'].str.strip()

## Cleaning club names

In [53]:
remove2 = ['AQUATIC', 'WATER POLO', 'CLUB', 'FEDERATION', 'TEAM', 'WP', 'WPC', 'POLO', 'POL', 'AQ', 'SUMMER', 
           'ORGANIZATION', 'ACADEMY', 'ORG', 'SWIM', 'AC', 'INC', 'FOUNDATION']
pat2 = r'\b(?:{})\b'.format('|'.join(remove2))
data_clean['Club'] = data_clean['Team'].str.replace(pat2, '').str.strip()
data_clean['Club'] = data_clean['Club'].str.replace('\s+', ' ', regex=True)

In [54]:
data_clean.loc[data_clean['Team'].str.contains('ACADEMY WATER POLO'), 'Club'] = 'ACADEMY'
data_clean.loc[data_clean['Team'] == 'EAST', 'Club'] = 'EAST'
data_clean['Club'] = data_clean['Club'].fillna('UNKNOWN')
data_clean.loc[data_clean['Team'] == '', 'Club'] = 'UNKNOWN'
data_clean['Club'] = data_clean['Club'].replace('ORANGECOUNTY', 'ORANGE COUNTY')

In [55]:
data_clean['Club'].value_counts().sort_index()

101                             1
14 SPRING                       1
16 SPRING                       1
209                             6
222                             1
241                             3
616                             1
680                           141
680 DRIVER                     15
680A                            1
A2                              3
ACADEMY                         6
ADMIRAL                        14
AETO                            2
AGWP                            1
AGWP NAVY                       1
ALAMEDA                         4
ALBUQUERQUE                     1
ALISO                           3
ALLIANCE                       31
ALUMNI                          1
AM RIV                          1
AM RIVER                       12
AMERICAN RIVER                 98
ANN ARBOR                       2
ANTEATER                        9
APACHE                          1
ARC                             1
ARROYO GRANDE                  21
ASA           

In [56]:
data_clean[data_clean['Club'].str.contains('COUNTY')]

,Finish,Team,Age,Sex,Year,og_name,Squad,Club
38,9,ORANGECOUNTY WATER POLO CLUB,14,Boys,2016,ORANGE COUNTY WATER POLO CLUB,A,ORANGE COUNTY
52,12,ORANGECOUNTY WATER POLO CLUB,18,Boys,2016,ORANGE COUNTY WATER POLO CLUB,A,ORANGE COUNTY
55,13,ORANGECOUNTY WATER POLO CLUB,16,Boys,2016,ORANGE COUNTY WATER POLO CLUB,A,ORANGE COUNTY
109,27,ORANGECOUNTY WATER POLO CLUB,12,Boys,2016,ORANGE COUNTY WATER POLO CLUB,A,ORANGE COUNTY
221,57,ORANGECOUNTY WATER POLO CLUB,14,Boys,2016,ORANGE COUNTY WATER POLO CLUB,A,ORANGE COUNTY
285,78,ORANGECOUNTY WATER POLO CLUB,16,Boys,2016,ORANGE COUNTY WATER POLO CLUB,A,ORANGE COUNTY
339,96,SOUTH COUNTYHERRIMAN POLO,16,Boys,2016,SOUTH COUNTY-HERRIMAN POLO,A,SOUTH COUNTYHERRIMAN
340,96,SOUTH COUNTYHERRIMAN POLO,18,Boys,2016,SOUTH COUNTY-HERRIMAN POLO,A,SOUTH COUNTYHERRIMAN
343,99,ORANGECOUNTY RIPTIDE,16,Boys,2016,ORANGE COUNTY RIPTIDE,A,ORANGECOUNTY RIPTIDE
10,3,ORANGECOUNTY WATER POLO CLUB,10,Mixed,2013,ORANGE COUNTY WATER POLO CLUB,A,ORANGE COUNTY


In [57]:
data_clean.loc[data_clean['og_name'].str.contains('TIED'), 'Tie'] = 1
data_clean['Tie'] = data_clean['Tie'].fillna(0)

In [58]:
data_clean

,Finish,Team,Age,Sex,Year,og_name,Squad,Club,Tie
0,1,LAGUNA BEACH WATER POLO,10,Girls,2016,LAGUNA BEACH WATER POLO,A,LAGUNA BEACH,0.0
1,1,VANGUARD AQUATIC,12,Boys,2016,VANGUARD AQUATICS,A,VANGUARD,0.0
2,1,LO ANGELE PREMIER,14,Boys,2016,LOS ANGELES PREMIER,A,LO ANGELE PREMIER,0.0
3,1,STANFORD WATER POLO FOUNDATION,16,Boys,2016,STANFORD WATER POLO FOUNDATION,A,STANFORD,0.0
4,1,REGENCY WATER POLO CLUB,18,Boys,2016,REGENCY WATER POLO CLUB,A,REGENCY,0.0
5,2,SANTA BARBARA PREMIER,10,Girls,2016,SANTA BARBARA PREMIER,A,SANTA BARBARA PREMIER,0.0
6,2,CC UNITED,12,Boys,2016,CC UNITED,A,CC UNITED,0.0
7,2,SOCAL WATER POLO FOUNDATION,14,Boys,2016,SOCAL WATER POLO FOUNDATION,A,SOCAL,0.0
8,2,SLEEPY HOLLOW AQUATIC,16,Boys,2016,SLEEPY HOLLOW AQUATICS,A,SLEEPY HOLLOW,0.0
9,2,680 WATER POLO,18,Boys,2016,680 WATER POLO,A,680,0.0


In [59]:
data_clean[data_clean['og_name'].str.contains('\r')]

,Finish,Team,Age,Sex,Year,og_name,Squad,Club,Tie


In [60]:
data_clean.to_csv('ranking_data.csv')